In [ ]:
from PIL import Image

# Open the image
img = Image.open('house.jpg')

# Coordinates of the doorknob (example values, you'll need to find these)
doorknob_coords = (x, y)

# Modify the pixel at the doorknob coordinates to red
img.putpixel(doorknob_coords, (255, 0, 0))

# Save the modified image
img.save('house_with_red_doorknob.jpg')
